Neurons from a layer are usually connected to neurons in the subsequent layer. What if we modify this so that the neurons also have
- self-connections
- connections latererally to every other neuron in its own layer ?

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Load Iris data
iris = load_iris()
X = iris.data
y = iris.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert arrays to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# Create datasets for DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False)

In [109]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        # Neurons' connections and biases
        self.neuron_connections_1 = nn.Parameter(torch.randn(4, 10))  # Connections from input to hidden neurons
        self.neuron_biases_1 = nn.Parameter(torch.zeros(10))           # Biases of hidden neurons

        self.neuron_connections_2 = nn.Parameter(torch.randn(10, 3))   # Connections from hidden to output neurons
        self.neuron_biases_2 = nn.Parameter(torch.zeros(3))            # Biases of output neurons

    def forward(self, x):
        # Forward pass through hidden neurons
        x = torch.mm(x, self.neuron_connections_1) + self.neuron_biases_1
        x = torch.relu(x)  # Activation function for hidden neurons

        # Forward pass through output neurons
        x = torch.mm(x, self.neuron_connections_2) + self.neuron_biases_2
        return x

In [110]:
model = IrisNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 15

for epoch in range(epochs):
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if (epoch+1) % 5 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 5/15, Loss: 0.1768
Epoch 10/15, Loss: 0.1990
Epoch 15/15, Loss: 0.1507


In [111]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test set: {100 * correct / total:.2f}%')

Accuracy of the network on the test set: 96.67%


In [112]:
model = None

In [113]:
class IrisNetModified(nn.Module):
    def __init__(self):
        super(IrisNetModified, self).__init__()
        self.input_to_hidden = nn.Parameter(torch.randn(4, 10))
        self.hidden_biases = nn.Parameter(torch.zeros(10))
        # Lateral connections within the hidden layer
        self.hidden_to_hidden = nn.Parameter(torch.randn(10, 10))  # Full matrix for intra-layer connections

        self.hidden_to_output = nn.Parameter(torch.randn(10, 3))
        self.output_biases = nn.Parameter(torch.zeros(3))
        # Lateral connections within the output layer
        self.output_to_output = nn.Parameter(torch.randn(3, 3))  # Full matrix for intra-layer connections

    def forward(self, x):
        # Input to hidden layer processing
        x = torch.mm(x, self.input_to_hidden) + self.hidden_biases
        # x = torch.relu(x)

        # Applying lateral connections within the hidden layer
        hidden_lateral = torch.mm(x, self.hidden_to_hidden)
        x = torch.relu(x + hidden_lateral)  # Combine initial activation with lateral effects

        # Hidden to output layer processing
        x = torch.mm(x, self.hidden_to_output) + self.output_biases
        # Applying lateral connections within the output layer
        output_lateral = torch.mm(x, self.output_to_output)
        x = x + output_lateral  # Combine output with lateral effects

        return x

In [114]:
model_modified = IrisNetModified()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_modified.parameters(), lr=0.01)

epochs = 15

for epoch in range(epochs):
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model_modified(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if (epoch+1) % 5 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 5/15, Loss: 0.3672
Epoch 10/15, Loss: 0.1416
Epoch 15/15, Loss: 0.0553


In [115]:
model_modified.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model_modified(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test set: {100 * correct / total:.2f}%')

Accuracy of the network on the test set: 100.00%
